In [ ]:
# %pip install -r requirements.txt

In [2]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from jinja2 import Template

from dotenv import load_dotenv
import json
import os
import json
import random

In [3]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
%pwd

'c:\\Users\\user\\Documents\\pja_MLOps\\Cat-Feat_recommend'

In [5]:
JSON_FILE_PATH = "data\\project_summary.json"
RCMD_PROMPT_PATH = "cat-feat_recommend.md"

In [13]:
# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()

In [11]:
json_data

[{'project_info_id': 1,
  'category': '웹 서비스',
  'core_features': ['ERD 자동생성', 'API 명세서 자동생성', '프로젝트 정보 자동생성'],
  'created_at': '2025-06-18T00:39:46.947095',
  'problem_solving': {'solutionIdea': 'AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화',
   'currentProblem': '초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음',
   'expectedBenefits': ['빠른 개발 착수', '효율적인 팀원 간 커뮤니케이션', '체계적인 프로젝트 경험']},
  'target_users': ['프로젝트 경험이 적은 초보 개발자'],
  'technology_stack': ['react',
   'typescript',
   'springboot',
   'AWS',
   'git action',
   'vercel'],
  'title': '프로젝트 관리 웹 서비스',
  'updated_at': '2025-06-18T00:39:46.947102',
  'workspace_id': 1},
 {'project_info_id': 11,
  'category': '학습 관리',
  'core_features': ['체크리스트 형태의 과제 관리', '퀴즈를 통한 반복 학습'],
  'created_at': '2025-06-19T08:24:29.404944',
  'problem_solving': {'solutionIdea': '스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공',
   'currentProblem': '학생들이 학습 동기를 잃고 효과적으로 학습하지 못하는 문제',
   'expectedBenefits': ['학습 효과 극대화', '사용자 간 소통 강화', '학습 동기 부여']},
  'target_users': ['학습

In [14]:
RCMD_PROMPT

'test prompt'

In [16]:
# project ID 추출
def extract_values_by_key(obj, key):
    values = []

    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == key:
                values.append(v)
            values.extend(extract_values_by_key(v, key))
    elif isinstance(obj, list):
        for item in obj:
            values.extend(extract_values_by_key(item, key))

    return values

# Read target file
with open(JSON_FILE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

target_key = "project_info_id"
proj_id_list = extract_values_by_key(data, target_key)

print(proj_id_list)

[1, 11, 4, 12, 6, 16, 15]


In [26]:
project_id = int(random.choice(proj_id_list))

def extract_workspace_view(input_json, id):
    # peject_id로 프로젝트 개요 읽어오기
    for item in input_json:
        if item.get("project_info_id") == id:
            return item
    return None

# peject_id로 읽어온 프로젝트 개요
project_summary = extract_workspace_view(data, project_id)
# project_summary  # test
type(project_summary)  # dict

if project_summary:
    print(json.dumps(project_summary, ensure_ascii=False, indent=2))  # <class 'str'>
else:
    print("** 해당 project_id를 찾을 수 없습니다. **")

{
  "project_info_id": 6,
  "category": "교육/학습 관리",
  "core_features": [
    "체크 리스트 형태의 과제 관리",
    "퀴즈를 통한 반복 학습"
  ],
  "created_at": "2025-06-19T02:01:30.915498",
  "problem_solving": {
    "solutionIdea": "스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 효과를 극대화",
    "currentProblem": "학생들이 학습 동기를 유지하기 어려움",
    "expectedBenefits": [
      "사용자 간의 소통 강화",
      "학습 동기 부여"
    ]
  },
  "target_users": [
    "학습 동기를 얻고 싶은 학생"
  ],
  "technology_stack": [
    "javaScript",
    "java"
  ],
  "title": "스터디 그룹 운영 지원 플랫폼",
  "updated_at": "2025-06-19T08:08:47.016643",
  "workspace_id": 8
}


In [27]:
# OpenAI 객체 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [ ]:
# 변수{} 삽입 가능한 템플릿 생성
prompt = PromptTemplate(
    input_variables=["input", "tools", "tool_names", "agent_scratched"],
    template=RCMD_PROMPT
    # template=template
)

In [ ]:
# llm(OpenAI) <-> Langchain 연결
llm_chain = LLMChain(prompt=prompt, llm=llm)

# LLM으로 category, feature, action 추천 받는 함수
def generate_next_actions(input_dict):
    return llm_chain.run(input_dict)

# 한 프로젝트의 요약 정보를 받아 추천 작업을 생성하는 메인 함수
# @tool(description="워크스페이스 JSON으로부터 feature별 추천 작업을 생성합니다.")
def parse_and_generate_all(proj_sum: dict) -> str:

    data = input_str  # 수정!!!!!!!

    # 웹 검색 툴 정의
    search = DuckDuckGoSearchRun()

    for category in data.get("recommendedCategories", []):  # recommendedCategories 가져오기
        for feature in category.get("features", []):        # recommendedCategories 내부의 feature에서

            # 웹 검색
            try:
                search_query = f"{feature['name']} 기능 구현 방법"  # feature name에 대한 actions 서치
                search_result = search.run(search_query)
            except Exception as e:
                search_result = f"** 웹 검색 실패: {e} **"
                
            # featureName, existingActions 변수 사용 안 하고 서치
            llm_input = search_result[:500]
            
            generated_actions = []

            try:
                generated_json_str = generate_next_actions(llm_input)  # 추천 actions 생성
                generated_actions = json.loads(generated_json_str)     # 추천 actions를 json형식의 python 객체로 load
                feature["actions"].extend(generated_actions)
            except Exception as e:
                feature["actions"].append({
                    "name": f"작업 생성 실패: {e}",
                    "startDate": "2025-01-01T00:00:00",
                    "endDate": "2025-01-01T23:59:59",
                    "importance": 0
                })

    # return json.dumps(data, ensure_ascii=False, indent=2)
    return data